### Logistic Regression - Label Classification

In [1]:
import sys
sys.path.append('../../../../')

# Import libraries
from utils.score import fold_cross_validate
from utils.print import print_cross_validation_scores
from constant.columns import FEATURES, LABEL, SIZE, TARGET_Y, MODE, IS_SINGLE_INPUT

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
# Import data
feat = FEATURES
if IS_SINGLE_INPUT ==  False:
    WINDOW_SIZE = f"{SIZE}_{TARGET_Y}"
    if MODE == 1:
        WINDOW_SIZE = f"{SIZE}_{TARGET_Y}_normalize"
    if MODE == 2:
        WINDOW_SIZE = f"{SIZE}_{TARGET_Y}_normalize_std"

    print(f'WINDOW_SIZE: {WINDOW_SIZE}')
    # Get features with window size
    expanded_features = []
    for feature in FEATURES:
        for i in range(SIZE):
            expanded_features.append(f'{feature}_{i}')
    
    feat = expanded_features

# # Read dataset
# df = pd.read_csv(f'./source/dataset_smote.csv')
# X, y = df[feat], df[LABEL]

# # Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
    
# Read dataset
df = pd.read_csv(f'./source/dataset_smote.csv')
X_train, y_train = df[feat], df[LABEL]

# Read test dataset
df_test = pd.read_csv(f'./source/dataset_test.csv')
X_test, y_test = df_test[feat], df_test[LABEL]

WINDOW_SIZE: 10_10


#### Cross Validation

In [5]:
from sklearn.metrics import f1_score

# Define the model
clf_cv = LogisticRegression()

# Train the model with SMOTE data
clf_cv.fit(X_train, y_train)

# Use the trained model to make predictions on the test set
y_pred = clf_cv.predict(X_test)

# Calculate the macro-averaged F1 score
f1_macro = f1_score(y_test, y_pred, average='macro')

# Print the macro-averaged F1 score
print(f"Macro-averaged F1 score: {f1_macro}")

Macro-averaged F1 score: 0.7498004105839415


c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
# # Define the optimized model (previous research)
clf_cv = LogisticRegression(
    penalty="l1", 
    C=0.1, 
    solver="liblinear", 
    max_iter=100, 
    class_weight=None, 
    fit_intercept=False, 
    multi_class="auto"
)

clf_cv.fit(X_train, y_train)

# Use the trained model to make predictions on the test set
y_pred = clf_cv.predict(X_test)

# Calculate the macro-averaged F1 score
f1_macro = f1_score(y_test, y_pred, average='macro')

# Print the macro-averaged F1 score
print(f"Macro-averaged F1 score: {f1_macro}")

c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Macro-averaged F1 score: 0.7771724963425988


c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Hyperparameter Tuning

In [14]:
from skopt import BayesSearchCV
from skopt.space import Categorical, Integer
from sklearn.linear_model import LogisticRegression

# Define the hyperparameter space
hyperparameters = {
    "penalty": Categorical(["l1", "l2"]),
    "C": Integer(1, 100),
    "solver": Categorical(["liblinear", "saga"]),
    "max_iter": Integer(100, 1000),
    "class_weight": Categorical([None, "balanced"]),
    "fit_intercept": Categorical([True, False]),
    "multi_class": Categorical(["auto", "ovr", "multinomial"]),
}

# Define the model
clf = LogisticRegression()

# Define the optimizer
opt = BayesSearchCV(
    clf,
    hyperparameters,
    n_iter=64,
    cv=10,
    scoring="f1_macro",
    random_state=0,
)

# Train the optimizer
opt.fit(X_train, y_train)

# Print the best hyperparameters
print(opt.best_params_)
print(opt.best_score_)

c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConve

ValueError: Solver liblinear does not support a multinomial backend.

In [ ]:
# Use the best model to make predictions on the test set
y_pred = opt.predict(X_test)

# Calculate the macro-averaged F1 score
f1_macro = f1_score(y_test, y_pred, average="macro")